In [1]:
from googleapiclient.discovery import build
import os

In [2]:
from keys import api_key, api_key2

In [3]:
from datetime import datetime, timedelta, timezone
import isodate

In [4]:
from datetime import datetime, timedelta
import pytz

In [5]:
youtube = build('youtube', 'v3', developerKey=api_key2)

In [6]:
def get_latest_video_id(channel_id):
    # Retrieve the list of videos uploaded to the channel.
    request = youtube.search().list(
        part="snippet",
        channelId=channel_id,
        maxResults=12,  # Increase the maxResults to retrieve more videos
        order="date",  # Order by date to get the latest video
        type="video"  # Look for videos only (no playlists or channels)
    )
    response = request.execute()
    
    for item in response.get('items', []):
        video_id = item['id']['videoId']
        # Make a request to get the video details
        video_request = youtube.videos().list(
            part="snippet,liveStreamingDetails,contentDetails",
            id=video_id
        )
        video_response = video_request.execute()
        
        # Check if there are items in the response
        if not video_response.get('items'):
            continue
        
        for i in range(len(video_response['items'])):
            video = video_response['items'][i]
            # Check if the video is not a livestream
            if 'liveStreamingDetails' not in video:
                # Get the video duration
                duration = video['contentDetails']['duration']
                # Convert the duration to seconds
                duration_seconds = isodate.parse_duration(duration).total_seconds()
                # Check if the video duration is greater than 60 seconds (not a YouTube Short)
                if duration_seconds > 70:
                    video_published_at = datetime.strptime(item['snippet']['publishedAt'], '%Y-%m-%dT%H:%M:%SZ')
                    video_published_at = video_published_at.replace(tzinfo=pytz.UTC)
                    now = datetime.now(pytz.UTC)
                    # Check if the video is not older than 1 day
                    if now - video_published_at <= timedelta(days=1):
                        return video_id
    
    # If no suitable video is found, return None
    return None

def get_latest_video_ids(video_ids, channel_ids):
    # Loop over channel IDs and append the video IDs to the list
    for channel_id in channel_ids:
        video_id = get_latest_video_id(channel_id)
        if video_id:
            video_ids.append(video_id)
            print(f"Added Video ID: {video_id}")



In [7]:

channel_ids = ['UCvJZEG5x-DVYZKTz--pS39w', 'UC7QHSNOKZvbqpXWFv0SiYDQ',"UCinBfsRWiOlzxg7DvjZ5mdQ", "UCUP_ao_7-Yct5FcVIA4Kobg", "UCAHJqpJPqqLLi7siVQpD-3A" , 'UCfdPOTevbfCh_QHsyPeZ8MQ',
               "UCYKtr6GfycBqQJf32tbQSbQ", 'UCpBPTKot6Im9SWs1upQMZKA', 'UCTeFsS-bP0XEt3NBMjfW2cA', 'UC-m6zNItyoDk5lSykDlhE4Q' , 'UCmlfS56nZsnX_5F1fjvX0QA', 'UCoAuxtwgh8LSHeMFqEwjgHA']  # List of example channel IDs
latest_video_ids = []
get_latest_video_ids(latest_video_ids, channel_ids)

print("Latest video IDs:", latest_video_ids)

Added Video ID: 7RZV45_hcM4
Added Video ID: uKQZbZsgzgg
Added Video ID: 0vZH6U4sdhU
Added Video ID: 6qM5goSBGVw
Added Video ID: 0Z_nqlGjxLk
Added Video ID: je25SULopko
Added Video ID: fhFWacpJCys
Added Video ID: rxnWbHNdd-4
Added Video ID: _3pt-gi33lk
Latest video IDs: ['7RZV45_hcM4', 'uKQZbZsgzgg', '0vZH6U4sdhU', '6qM5goSBGVw', '0Z_nqlGjxLk', 'je25SULopko', 'fhFWacpJCys', 'rxnWbHNdd-4', '_3pt-gi33lk']


In [8]:
latest_video_ids

['7RZV45_hcM4',
 'uKQZbZsgzgg',
 '0vZH6U4sdhU',
 '6qM5goSBGVw',
 '0Z_nqlGjxLk',
 'je25SULopko',
 'fhFWacpJCys',
 'rxnWbHNdd-4',
 '_3pt-gi33lk']

In [9]:
import youtube_dl

def get_latest_video_url(channel_urls):
    video_urls = []
    for channel_url in channel_urls:
        with youtube_dl.YoutubeDL({'quiet': True}) as ydl:
            result = ydl.extract_info(channel_url, download=False)
            if 'entries' in result:
                # Sort videos by upload date
                latest_video = sorted(result['entries'], key=lambda x: x['upload_date'], reverse=True)[0]
                video_urls.append(latest_video['webpage_url'])
    return video_urls


In [10]:
import feedparser
from datetime import datetime, timedelta

# Function to get the latest video ID and upload date from a YouTube channel using RSS feed
def get_latest_video_id(channel_id):
    # Construct the RSS feed URL for the YouTube channel
    feed_url = f"https://www.youtube.com/feeds/videos.xml?channel_id={channel_id}"
    # Parse the feed
    feed = feedparser.parse(feed_url)
    # Get the first entry, which is the latest video
    latest_video = feed.entries[0]
    # Parse the published date
    published_date = datetime.strptime(latest_video.published, '%Y-%m-%dT%H:%M:%S+00:00')
    # Check if the video is less than a day old
    if datetime.utcnow() - published_date <= timedelta(days=2):
        # Extract video ID from the URL
        video_id = latest_video.link.split("=")[-1]
        # Return the video ID if the video is within one day old
        return video_id
    else:
        return None


def get_latest_video_name(channel_id):
    # Construct the RSS feed URL for the YouTube channel
    feed_url = f"https://www.youtube.com/feeds/videos.xml?channel_id={channel_id}"
    # Parse the feed
    feed = feedparser.parse(feed_url)
    # Get the first entry, which is the latest video
    latest_video = feed.entries[0]
    # Parse the published date
    published_date = datetime.strptime(latest_video.published, '%Y-%m-%dT%H:%M:%S+00:00')
    # Check if the video is less than a day old
    if datetime.utcnow() - published_date <= timedelta(days=1):
        # Extract video ID from the URL
        title= latest_video.title
        # Return the video ID if the video is within one day old
        return title
    else:
        return None

In [11]:
latest_video_ids

['7RZV45_hcM4',
 'uKQZbZsgzgg',
 '0vZH6U4sdhU',
 '6qM5goSBGVw',
 '0Z_nqlGjxLk',
 'je25SULopko',
 'fhFWacpJCys',
 'rxnWbHNdd-4',
 '_3pt-gi33lk']

In [12]:
from youtube_transcript_api import YouTubeTranscriptApi


def get_youtube_captions(video_id):
    try:
        # Retrieve the available subtitles
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        current_minute = 0
        paragraph = []
        paragraphs = []
        
        # Process each subtitle line
        for item in transcript:
            # Calculate the minute mark of the current subtitle
            item_minute = int(item['start'] // 60)
            
            # Check if the current subtitle still falls within the current minute window
            if item_minute == current_minute:
                paragraph.append(item['text'])
            else:
                # Store the completed paragraph and start a new one
                paragraphs.append(' '.join(paragraph))
                paragraph = [item['text']]
                current_minute = item_minute
        
        # Add the last paragraph if any
        if paragraph:
            paragraphs.append(' '.join(paragraph))
        
        # Join all paragraphs with two newlines
        formatted_text = '\n\n'.join(paragraphs)
        
        # Optionally save the subtitles to a file
        with open(f'{video_id}_subtitles.txt', 'w', encoding='utf-8') as f:
            f.write(formatted_text)
        
        return formatted_text
    except Exception as e:
        return str(e)

# Assuming latest_video_ids is a list of video IDs
for video_id in latest_video_ids:
    captions = get_youtube_captions(video_id)
    print(captions)

well guys it looks like the media has done a full 180 and we're back to talking and discussing the idea that we're in 1929 but are we really there as Traders and investors and should we be worried today we share what Tom Lee believes will happen during the month of October which might surprise you and we take a look at the flow starting to come back into the world's biggest hedge the US dollar also what's going on in some of the markets that have been let's say not performing well in recent times it looks like breakouts could be on the horizon while the biggest short squeeze just had one of the largest trades ever recorded we suspected that China might be over for now maybe we just got the proof let's take a look at what sneaky Wall Street is up to right now we'll see you in a moment well welcome back everybody to The Daily Show where we discuss stocks Commodities and cryptos together my name is Tom and today we've got a lot of information to sort out and of course some big flows to go

In [13]:
'Analyze the content from the file I've provided, to answer: What is the short term forecast of the market, especially SPY and ETFs. Give me bullish cases or indicators, and bearish cases or indicators. The amount of bullet points should reflect the analyst's stance on whether they are bullish or bearish overall. Do not use today's returns as bullish or bearish.  


SyntaxError: invalid syntax (4132480804.py, line 1)

In [ ]:
'What is the monthly forecast based on historical evidence. Be specific about price action behavior.'

In [ ]:
"Give me a summary and investment advice based on this analysis. Be specific."

In [ ]:
'''what are the predicted price areas of note and key levels, short and long term (upper and lower bounds), and annotate the levels (IE: resistance, or order wall, etc.) try to be specific, especially for IWM and SPY
IE:
short term
lower: lower resistance
upper: price target

long term
lower: etc
upper: etc'''

In [ ]:
"What 3 options are optimal in this scenario given the analysts advice, give me price levels and expiration dates. This isn't asking for financial advice but a test on your understanding of the given content."

Do they think will there be a pullback soon? Use more explicity stated evidence and references.

Is there any evidence of explicit bullish or bearish sentiment in the text

What is the monthly forecast based on historical evidence. Be specific about price action behavior.

In [ ]:
video_id='8YQIdkFX8xA'

In [ ]:
get_youtube_captions(video_id)

In [ ]:
import openai

def get_chatgpt_response(prompt_file_path):
    # Load your API key from an environment variable or direct string (not recommended for production)
    openai.api_key = 'your-api-key-here'

    # Read the prompt from the file
    with open(prompt_file_path, 'r') as file:
        prompt = file.read()

    # Send the prompt to the ChatGPT model
    response = openai.ChatCompletion.create(
        model="gpt-4",  # or use "text-davinci-003" for GPT-3.5
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    # Return the response text
    return response.choices[0].message['content']

# Example usage
#prompt_path = 'path_to_your_prompt_file.txt'  # Change this to the path of your prompt file
#response_text = get_chatgpt_response(prompt_path)
#print("ChatGPT response:", response_text)